<a href="https://colab.research.google.com/github/jtlagumbay/cebqa/blob/main/reader/cebqa_roberta_body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CebQA Reader Component**
Pretrained model: RoBERTa

# **Libraries**

In [1]:
!pip install datasets
!pip install evaluate
# !pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
import pandas as pd
from datasets import Dataset, load_dataset
from evaluate import load
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaForQuestionAnswering, TrainingArguments, Trainer, XLMRobertaTokenizerFast, EarlyStoppingCallback, pipeline, AutoModelForQuestionAnswering, AutoTokenizer
from transformers.trainer_utils import get_last_checkpoint
from huggingface_hub import login
import datetime
from google.colab import drive
from IPython.display import display
from sklearn.metrics import f1_score
import re
# import optuna
import numpy as np
import unicodedata
from collections import defaultdict


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Constants**

In [4]:
CEBQA_DATASET = "jhoannarica/cebquad"
DRIVE_ROOT = "/content/drive/MyDrive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr_body-filtered"
OUTPUT_DIRECTORY = "training_output"
LOGS_DIRECTORY = "logs"
MODEL_DIRECTORY = "model"
TOKENIZER_DIRECTORY = "tokenizer"

# **Utils**

In [5]:
def timestamp():
  return datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")

def get_output_directory(batch_timestamp):
  return f"{DRIVE_ROOT}/{batch_timestamp}/{OUTPUT_DIRECTORY}"

def get_logs_directory(batch_timestamp):
  return f"{DRIVE_ROOT}/{batch_timestamp}/{LOGS_DIRECTORY}"

def get_model_directory(batch_timestamp):
  return f"{DRIVE_ROOT}/{batch_timestamp}/{MODEL_DIRECTORY}"

def get_tokenizer_directory(batch_timestamp):
  return f"{DRIVE_ROOT}/{batch_timestamp}/{TOKENIZER_DIRECTORY}"

# **Loading Dataset**

## Access dataset

In [6]:
dataset = load_dataset("jhoannarica/CebQuAD_fixed")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/6.07M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/11.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19340 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2752 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5526 [00:00<?, ? examples/s]

# **Prepare Dataset**

## Prepare tokenizer

## Tokenize

In [7]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [8]:
def filter_incomplete_examples(example):
    # Ensure both "question" and "context" exist and are non-empty
    return "question" in example and example["question"] and \
           "article_body" in example and example["answer"]

def filter_by_token_length(example):
    # Tokenize the concatenated question + article_body
    tokens = tokenizer(example["question"], example["article_body"], truncation=False)
    return len(tokens["input_ids"]) <= 512

def decode_error(example):
  input_ids = example["input_ids"]
  start_positions = example["start_positions"]
  end_positions = example["end_positions"]
  predict_answer_tokens = input_ids[start_positions : end_positions+1]
  return tokenizer.decode(predict_answer_tokens) == example["answer"]

In [9]:
def tokenize_train_function(examples):
    article_text = [article for article in examples.get("article_body", [""])]
    context_text = [context for context in examples.get("context", ["{}"])]
    answer_text = examples.get("answer", [""])
    answer_start = examples.get("answer_start", [0])
    context_start_list = examples.get("context_start", [0])
    question_text = [q for q in examples.get("question", [""])]
    start_positions = []
    end_positions = []

    inputs = tokenizer(
        question_text,
        article_text,
        truncation="only_second",  # Truncate only the context
        max_length=512,            # Limit input length
        stride=128,                # Add a sliding window
        return_overflowing_tokens=False,  # Handle long contexts
        return_offsets_mapping=True,
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    # sample_map = inputs.pop("overflow_to_sample_mapping")

    for i, offset in enumerate(offset_mapping):
        answer = answer_text[i]
        context = context_text[i]
        article = article_text[i]
        start_char = int(context_start_list[i]) + int(answer_start[i])
        end_char = start_char + len(answer)


        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions


    return inputs


In [12]:
def normalize_text(examples):
    examples["context"] = [unicodedata.normalize("NFKC", context) for context in examples["context"]]

    examples["article_body"] = [unicodedata.normalize("NFKC", body) for body in examples["article_body"]]

    examples["answer"] =  [unicodedata.normalize("NFKC", answer) for answer in examples["answer"]]

    examples["question"] = [unicodedata.normalize("NFKC", q) for q in examples["question"]]

    return examples


In [ ]:
dataset["train"]

In [13]:
# Clean and tokenize the dataset
tokenized_train_dataset = dataset["train"].filter(filter_incomplete_examples) \
  .map(normalize_text, batched=True) \
  .map(tokenize_train_function, batched=True)\
  .filter(decode_error)



Map:   0%|          | 0/19340 [00:00<?, ? examples/s]

Map:   0%|          | 0/19340 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19340 [00:00<?, ? examples/s]

In [14]:
# Clean and tokenize the dataset
tokenized_validation_dataset = dataset["validation"].filter(filter_incomplete_examples) \
  .map(normalize_text, batched=True) \
  .map(tokenize_train_function, batched=True)\
  .filter(decode_error)


Filter:   0%|          | 0/2752 [00:00<?, ? examples/s]

Map:   0%|          | 0/2751 [00:00<?, ? examples/s]

Map:   0%|          | 0/2751 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2751 [00:00<?, ? examples/s]

In [15]:
# Clean and tokenize the dataset
tokenized_test_dataset = dataset["test"].filter(filter_incomplete_examples) \
  .map(normalize_text, batched=True) \
  .map(tokenize_train_function, batched=True)\
  .filter(decode_error)


Filter:   0%|          | 0/5526 [00:00<?, ? examples/s]

Map:   0%|          | 0/5526 [00:00<?, ? examples/s]

Map:   0%|          | 0/5526 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5526 [00:00<?, ? examples/s]

In [14]:
print(f"train: {len(tokenized_train_dataset)} validate: {len(tokenized_validation_dataset)} ")

train: 14430 validate: 2234 


## Dataset Splitting

In [16]:
train_dataset = tokenized_train_dataset
val_dataset = tokenized_validation_dataset
test_dataset = tokenized_test_dataset

print(f"train: {train_dataset.num_rows} \nval: {val_dataset.num_rows} \ntest: {test_dataset.num_rows}")
# train_dataset[0]
# len(val_dataset)


train: 14430 
val: 2234 
test: 4141


In [77]:
def test_decode(dataset, idx):
  input_ids = dataset[idx]["input_ids"]
  start_positions = dataset[idx]["start_positions"]
  end_positions = dataset[idx]["end_positions"]
  predict_answer_tokens = input_ids[start_positions : end_positions+1]
  return tokenizer.decode(predict_answer_tokens), dataset[idx]["answer"]

# idx = 12
# print(test_decode(train_dataset, idx))
# print(test_decode(val_dataset, idx))
error_id = []
for idx, train in enumerate(val_dataset):
    decoded, orig = test_decode(val_dataset, idx)
    if decoded != orig:
        # print(f"idx: {train_dataset[idx]['id']}")
        # print(val_dataset[idx]['id'])
        # print(val_dataset[idx]['start_positions'], val_dataset[idx]['end_positions'])
        if not (val_dataset[idx]['start_positions'] == 0 and val_dataset[idx]['end_positions']== 0):
            error_id.append(val_dataset[idx]['id'])

    # if idx == 100:
    #   break
print(len(error_id))

error_id = []
for idx, train in enumerate(train_dataset):
    decoded, orig = test_decode(train_dataset, idx)
    if decoded != orig:
        # print(f"idx: {train_dataset[idx]['id']}")
        # print(val_dataset[idx]['id'])
        # print(val_dataset[idx]['start_positions'], val_dataset[idx]['end_positions'])
        # if not (train_dataset[idx]['start_positions'] == 0 and train_dataset[idx]['end_positions']== 0):
        error_id.append(train_dataset[idx]['id'])

    # if idx == 100:
    #   break
print(len(error_id))

133
4910


In [64]:
print(len(error_id))

132


In [21]:
print(error_id)

['00371-003', '01200-009', '00943-005', '00419-017', '01193-007', '00278-009', '00599-011', '00395-011', '00077-009', '01283-014', '00826-016', '01496-014', '00265-012', '00498-017', '00521-011', '01675-009', '00558-013', '00337-010', '00948-009', '01254-006', '01283-016', '00228-013', '01448-014', '00383-010', '00179-017', '00389-011', '01634-011', '00326-003', '01650-015', '00756-005', '01689-009', '00095-006', '00464-012', '00906-009', '01564-009', '01002-012', '00952-017', '00485-011', '01601-009', '01595-006', '00621-009', '01260-015', '00329-014', '00334-017', '01319-014', '00914-011', '01412-007', '01041-016', '00548-015', '00065-012', '00413-015', '00332-017', '00673-011', '01356-001', '00297-011', '00916-017', '01236-009', '00404-017', '01349-011', '00241-010', '00686-017', '01435-015', '01268-009', '00521-016', '00466-010', '00952-013', '01670-010', '00802-015', '00053-016', '00430-011', '01301-017', '00524-007', '00053-012', '01416-004', '01187-009', '01670-009', '00834-009'

# **Model Training**

## Compute Metrics

In [44]:
metric = load("squad")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    start_preds = np.argmax(predictions[0], axis=1)
    end_preds = np.argmax(predictions[1], axis=1)

    decoded_preds = [
        tokenizer.decode(input_ids[start : end+1])
        for input_ids, start, end in zip(val_dataset["input_ids"], start_preds, end_preds)
    ]

    decoded_labels = [
        tokenizer.decode(input_ids[start:end+1], skip_special_tokens=True)
        for input_ids, start, end in zip(val_dataset["input_ids"], labels[0], labels[1])
    ]
    pred = [{"prediction_text": pred, "id": str(i)} for i, pred in enumerate(decoded_preds)]
    ref = [{"answers": {"text": [label], "answer_start": [0]}, "id": str(i)} for i, label in enumerate(decoded_labels)]

    results = metric.compute(
        predictions=pred,
        references=ref
    )
    sentence_match_scores = [
        p['prediction_text'] in r['answers']['text'][0] for p, r in zip(pred, ref)
    ]

    # Compute average sentence match score
    avg_sentence_match = np.mean(sentence_match_scores)

    res = {
        "exact_match": results["exact_match"],
        "f1": results["f1"],
        "sentence_match": float(avg_sentence_match ) * 100
    }

    return res



## Finetuning

In [37]:
def finetune_xlmr(
    model_path = "xlm-roberta-large",
    checkpoint_path = None,
    learning_rate = 1e-5,
    batch_size = 8,
    num_train_epochs = 1,
    weight_decay = 0.01
    ):
    model = XLMRobertaForQuestionAnswering.from_pretrained(model_path)

    # Early stopping parameters
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=3,
        early_stopping_threshold=0.1
    )

    batch_timestamp = timestamp()
    print(batch_timestamp)

    last_checkpoint = get_last_checkpoint(checkpoint_path) if checkpoint_path else None

    # Define training arguments with suggested values
    training_args = TrainingArguments(
        output_dir=get_output_directory(batch_timestamp),
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        learning_rate=learning_rate,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        logging_dir=get_logs_directory(batch_timestamp),
        logging_steps=10,
        save_total_limit=2,
        bf16=True,
        metric_for_best_model="eval_f1",
        greater_is_better=True
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_callback]
    )

      # Train and evaluate the model
    trainer.train(
        resume_from_checkpoint=last_checkpoint if last_checkpoint else None
        )
    eval_results = trainer.evaluate()

    res_model_path = get_model_directory(batch_timestamp)
    res_tokenized_path = get_tokenizer_directory(batch_timestamp)

    model.save_pretrained(res_model_path)
    tokenizer.save_pretrained(res_tokenized_path)

    return model, tokenizer, eval_results, res_model_path, res_tokenized_path

## Optuna

In [ ]:

# Suggest values for hyperparameters in Optuna
# learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-4)
# batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
# num_train_epochs = trial.suggest_int("num_train_epochs", 2, 5)
# weight_decay = trial.suggest_loguniform("weight_decay", 1e-3, 0.1)

study = optuna.create_study(direction="maximize")
study.optimize(finetune_xlmr, n_trials=1)

# Get the best trial
best_trial = study.best_trial
# Print best trial number and its hyperparameters
print(f"Best Trial: {best_trial.number}")
print("Best Hyperparameters:", best_trial.params)
print(f"Best F1 Score: {best_trial.value:.4f}")

[I 2025-03-10 13:44:01,276] A new study created in memory with name: no-name-525be363-eb88-442a-80fd-1fb72ab389ac


2025-03-10_13-44-0


Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-46-ac82ea0702a3>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-4)
<ipython-input-46-ac82ea0702a3>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-3, 0.1)


Epoch,Training Loss,Validation Loss


## Start Training

In [52]:
model, tokenizer, eval_results, res_model_path, res_tokenized_path = finetune_xlmr(
    num_train_epochs  = 5,
    batch_size = 32,
)
print(eval_results)
print(res_model_path)
print(res_tokenized_path)

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2025-04-01_13-27


Epoch,Training Loss,Validation Loss,Exact Match,F1,Sentence Match
1,1.477100,1.429074,48.970457,68.325351,69.292748
2,1.093600,1.141033,56.580125,75.741076,73.634736
3,1.008600,1.109945,57.520143,76.437348,73.410922
4,0.697800,1.152499,58.012534,77.138168,72.291853
5,0.572100,1.222715,57.967771,77.540310,73.321397


{'eval_loss': 1.2227146625518799, 'eval_exact_match': 57.96777081468218, 'eval_f1': 77.54031002785501, 'eval_sentence_match': 73.32139659803045, 'eval_runtime': 8.8903, 'eval_samples_per_second': 251.286, 'eval_steps_per_second': 7.874, 'epoch': 5.0}
/content/drive/MyDrive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr_body-filtered/2025-04-01_13-27/model
/content/drive/MyDrive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr_body-filtered/2025-04-01_13-27/tokenizer


# **Evaluating the model**

## Normalizing predicted answer

In [39]:
def normalize_row(text):
    """Lowercase and remove punctuation, articles, and extra whitespace."""
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)  # Remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

## Evaluating

In [53]:
def evaluate(model, tokenizer, dataset, model_outputs=None):
  print(f"Started evaluation.")

  if model_outputs is None:
      qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

      print("Generated QA Pipeline.")
      print("Starting QA Pipeline batch.")
      qa_dataset = Dataset.from_dict({
        "question": [sample["question"] for sample in dataset],
        "context": [sample["article_body"] for sample in dataset]
      })

      model_outputs = qa_pipeline(qa_dataset)

  print(f"Batched QA done. {len(model_outputs)}")
  print(f"Computing metrics.")
  print("Before pred")
  print(model_outputs[0])
  pred = [
      {
          'id': str(i+1),  # Convert ID to string
          'prediction_text': normalize_row(output['answer'])
      }
      for i, output in enumerate(model_outputs)
  ]
  print("Before ref")
  ref = [
      {
          'id': str(i+1),  # Convert ID to string
          'answers': {
              'text': normalize_row(row['answer']) if isinstance(row['answer'], list) else ([normalize_row(row['answer'])]),
              'answer_start': row['answer_start'] if isinstance(row['answer_start'], list) else [row['answer_start']]
          }
      }
      for i, row in enumerate(dataset)
  ]

  # Load SQuAD metric
  metric = load("squad")

  # Compute metric
  res = metric.compute(predictions=pred, references=ref)
  print(f"Computing metrics done.")

  # Sentence match
  sentence_match_scores = [
      p['prediction_text'] in r['answers']['text'][0] for p, r in zip(pred, ref)
  ]

  # Compute average sentence match score
  avg_sentence_match = np.mean(sentence_match_scores)

  # Combine results
  res["sentence_match"] = float(avg_sentence_match ) * 100

  return res, pred, ref, model_outputs


In [54]:
useExisting = False
if useExisting or (model == None and tokenizer == None):
    print("Loading Previous")
    model_path = DRIVE_ROOT+"/2025-04-01_05-56/model"
    tokenizer_path = DRIVE_ROOT+"/2025-04-01_05-56/tokenizer"
    model = AutoModelForQuestionAnswering.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

eval_results = evaluate(
    model = model,
    tokenizer = tokenizer,
    dataset = test_dataset
)

Device set to use cuda:0


Started evaluation.
Generated QA Pipeline.
Starting QA Pipeline batch.


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Batched QA done. 4141
Computing metrics.
Before pred
{'score': 0.9342123866081238, 'start': 431, 'end': 440, 'answer': 'Imee Niu.'}
Before ref
Computing metrics done.


## **Reminder**

1. Save Data to [spreadsheet](https://docs.google.com/spreadsheets/d/1Xc3-6yVMMLoXCqId-YQFyOvgQfemIQ8P2uLYC6Jszeo/edit?gid=0#gid=0)
2. Save WANDB chart to [Google Drive](https://drive.google.com/drive/u/0/folders/1inDiei-xuRlofFPJmVj8OS6pmyQdGQ2z) of the corresponding model.
3. Change runtime after.

In [55]:
res, pred, ref, model_outputs = eval_results

display(pd.DataFrame(res, index=[0]))




,exact_match,f1,sentence_match
0,54.793528,72.318191,70.8766


In [ ]:
pred_answers = [ans["prediction_text"] for ans in pred]
ref_answers = [ans["answers"]["text"][0] for ans in ref]

df = pd.DataFrame({
    "Predicted Answer": pred_answers,
    "Reference Answer": ref_answers
})


display(df)

,Predicted Answer,Reference Answer
0,imee niu,si imee niu
1,si mabansag,dali nga nakasibat
2,tulo ka dagkong mga distrito,south road properties srp new business and hou...
3,nalambigit sa illegal drugs,nagsugod sa balay sa usa ka juling pamugas nga...
4,molurang ang dautang panahon,kon molurang ang dautang panahon
...,...,...
5521,seaborne procession,ang mga sakay gidali sa pagtabang sa laing tag...
5522,nidagan kini subay sa dan katipunan,ginadiling drugas
5523,ritual showdown best musical ensemble ug best ...,canlaon city sa negros oriental ug lungsod sa ...
5524,police regional office 7,udto sa domingo agusto 25 2024 hangtod nga nat...
